In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import os 

from utils_models.utils_functions import canonical_form, normalize_features, get_model_matrices

In [2]:
project_root = os.path.dirname(os.getcwd())
problem = os.path.join(project_root, 'data/GAMS_library', 'AIRSP.mps')
os.path.exists(problem)

True

In [3]:
model = gp.read(problem)
original_primal, track_elements = canonical_form(model)
print(type(original_primal))
original_primal.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-11
Read MPS format model from file /home/brgonzalez/Escritorio/TESIS/Code/Interpretable_Optimization/data/GAMS_library/AIRSP.mps
Reading time = 0.00 seconds
Convert: 36 rows, 49 columns, 129 nonzeros
<class 'gurobipy.Model'>
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 43 rows, 51 columns and 197 nonzeros
Model fingerprint: 0xcf428e73
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Presolve removed 19 rows and 14 columns
Presolve time: 0.00s
Presolved: 24 rows, 37 columns, 102 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4486615e+

In [4]:
if model.Status == GRB.OPTIMAL:
    print('Optimal objective value:', model.ObjVal)
    # Get the variable names and values
    variables = model.getVars()
    var_names = [v.VarName for v in variables]
    var_values = model.getAttr('X', variables)
    # Iterate over names and values using zip
    for var_name, value in zip(var_names, var_values):
        print(f'{var_name}: {value}')
elif model.Status == GRB.INFEASIBLE:
    print('Model is infeasible.')
elif model.Status == GRB.UNBOUNDED:
    print('Model is unbounded.')
else:
    print('Optimization ended with status %d' % model.Status)


Optimization ended with status 1


Resolve normalize

In [5]:
def build_normalized_model(A_norm, b_norm, c, lb, ub, of_sense, cons_senses, var_names):
    """
    Builds a Gurobi model using the normalized A and b, and original c, lb, ub.
    """
    model = gp.Model()

    # Set objective sense
    model.ModelSense = of_sense  # 1 for minimization, -1 for maximization

    # Add variables
    num_vars = len(c)
    variables = []
    for i in range(num_vars):
        var = model.addVar(lb=lb[i], ub=ub[i], obj=c[i], name=var_names[i])
        variables.append(var)
    model.update()

    # Add constraints
    num_constrs = len(b_norm)
    for i in range(num_constrs):
        expr = gp.LinExpr()
        for j in range(num_vars):
            if A_norm[i, j] != 0:
                expr.add(variables[j], A_norm[i, j])
        sense = cons_senses[i]
        rhs = b_norm[i]
        if sense == '<':
            model.addConstr(expr <= rhs)
        elif sense == '>':
            model.addConstr(expr >= rhs)
        elif sense == '=':
            model.addConstr(expr == rhs)
        else:
            raise ValueError(f"Unknown constraint sense: {sense}")
    model.update()

    return model


In [6]:
A, b, c, co, lb, ub, of_sense, cons_senses, variables_names = get_model_matrices(original_primal)
A_norm, b_norm, scalers = normalize_features(A, b) 
# Run the model with normalized features
normalized_model = build_normalized_model(A_norm, b_norm, c, lb, ub, of_sense, cons_senses, variables_names)

# Optimize the normalized model
normalized_model.optimize()

# Retrieve and print results
if normalized_model.Status == GRB.OPTIMAL:
    print('Optimal objective value (normalized model):', normalized_model.ObjVal)
    solution = normalized_model.getAttr('X', normalized_model.getVars())
    for var_name, value in zip(variables_names, solution):
        print(f'{var_name}: {value}')
else:
    print('Optimization was not successful.')

# # Optional: Compare with the original model
# model.optimize()
# if model.Status == GRB.OPTIMAL:
#     print('Optimal objective value (original model):', model.ObjVal)
# else:
#     print('Original model optimization was not successful.')


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 43 rows, 51 columns and 197 nonzeros
Model fingerprint: 0x9ae832c4
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Presolve removed 19 rows and 14 columns
Presolve time: 0.00s
Presolved: 24 rows, 37 columns, 102 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4486615e+02   1.318648e+02   0.000000e+00      0s
      22    1.5660422e+03   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.566042189e+03
Optimal objective value (normalized model): 1566.0421891327062
x1: 10.0
x2: 0.0
x3: 0.0
x4: 0.0
x5: 0.0
x6: 12.84